In [ ]:
"""
Autoformer: Enhancing Transformers for Time Series Forecasting

Autoformer improves Transformers for time series forecasting by introducing two key innovations:

1. **Decomposition of Input Series**: The input time series is decomposed into trend and seasonality components. This allows the model to focus on learning each component separately, improving its ability to capture long-term dependencies and patterns. ✂️

2. **Improved Long-Term Prediction**: By learning the trend and seasonality components independently, Autoformer enhances its forecasting accuracy, especially for long-term predictions. 📈

This implementation is designed to be clean, lightweight, and runnable, similar to the Informer model.
"""

In [1]:
# 📦 Step 1: Import Libraries
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os
import sys

sys.path.append('../')  # Handle custom imports

from models.timeseries_dataset_class import TimeSeriesDataset
from models.transformer_model_definitions import SeriesDecomposition, AutoformerBlock, AutoformerForecast

# ✅ Enable GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)

Using: cpu


In [2]:
# 📂 Step 2: Load Preprocessed Splits
train_df = pd.read_csv("../data/processed/etth1_train.csv", parse_dates=["date"], index_col="date")
val_df = pd.read_csv("../data/processed/etth1_val.csv", parse_dates=["date"], index_col="date")
test_df = pd.read_csv("../data/processed/etth1_test.csv", parse_dates=["date"], index_col="date")


In [3]:
#parameters

target_column = "OT"
window_size = 96
batch_size = 32
epochs = 10
learning_rate = 1e-3

In [4]:
# 🔄 Step 3: Create Datasets and Dataloaders
train_loader = DataLoader(TimeSeriesDataset(train_df[target_column].values, window_size),
                          batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TimeSeriesDataset(val_df[target_column].values, window_size),
                        batch_size=batch_size, shuffle=False)
test_loader = DataLoader(TimeSeriesDataset(test_df[target_column].values, window_size),
                         batch_size=batch_size, shuffle=False)

In [5]:

# ⚙️ Step 4: Train Autoformer Model
model = AutoformerForecast().to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device).unsqueeze(1)
        pred = model(x)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"[Epoch {epoch+1}] Train Loss: {total_loss / len(train_loader):.4f}")

[Epoch 1] Train Loss: 0.0302
[Epoch 2] Train Loss: 0.0007
[Epoch 3] Train Loss: 0.0006
[Epoch 4] Train Loss: 0.0006
[Epoch 5] Train Loss: 0.0006
[Epoch 6] Train Loss: 0.0006
[Epoch 7] Train Loss: 0.0006
[Epoch 8] Train Loss: 0.0006
[Epoch 9] Train Loss: 0.0005
[Epoch 10] Train Loss: 0.0006


In [6]:
# 💾 Save the trained model
os.makedirs("../models/checkpoints", exist_ok=True)
torch.save(model.state_dict(), "../models/checkpoints/autoformer_transformer_model.pth")
print("✅ Model saved")

✅ Model saved


In [7]:
# 📊 Step 5: Define Evaluation Function
def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return 100 * np.mean(diff)

In [8]:
def evaluate_model(dataloader, dataset_df, title, file_prefix):
    model.eval()
    all_preds, all_trues = [], []
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device).unsqueeze(1)
            pred = model(x)
            all_preds.append(pred.cpu().numpy())
            all_trues.append(y.cpu().numpy())

    predictions = np.concatenate(all_preds).flatten()
    true_values = np.concatenate(all_trues).flatten()

    # Inverse scale using original df
    dummy_shape = (predictions.shape[0], dataset_df.shape[1])
    predictions_full = np.zeros(dummy_shape)
    true_values_full = np.zeros(dummy_shape)
    predictions_full[:, -1] = predictions
    true_values_full[:, -1] = true_values

    scaler = MinMaxScaler()
    scaler.fit(dataset_df)
    predictions = scaler.inverse_transform(predictions_full)[:, -1]
    true_values = scaler.inverse_transform(true_values_full)[:, -1]

    # 💡 Metrics
    mse = mean_squared_error(true_values, predictions)
    mae = mean_absolute_error(true_values, predictions)
    smape_val = smape(true_values, predictions)

    print(f"📊 {title}")
    print(f" - MSE   : {mse:.6f}")
    print(f" - MAE   : {mae:.6f}")
    print(f" - SMAPE : {smape_val:.2f}%")

    # 📈 Plot
    os.makedirs("../../outputs/metrics/autoformer", exist_ok=True)
    plt.figure(figsize=(12, 6))
    plt.plot(true_values[:100], label="True")
    plt.plot(predictions[:100], label="Predicted")
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.savefig(f"../../outputs/metrics/autoformer/{file_prefix}_plot.png")
    print(f"✅ Plot saved to ../../outputs/metrics/autoformer/{file_prefix}_plot.png")
    plt.close()

    # 💾 Save results
    pd.DataFrame({
        "True Values": true_values,
        "Predictions": predictions
    }).to_csv(f"../../outputs/metrics/autoformer/{file_prefix}_results.csv", index=False)
    print(f"✅ Results saved to ../../outputs/metrics/autoformer/{file_prefix}_results.csv\n")



In [9]:
# 📈 Step 6: Evaluate on All Splits
evaluate_model(train_loader, train_df, "📈 Train Set Forecast vs True", "train")
evaluate_model(val_loader, val_df, "📈 Validation Set Forecast vs True", "val")
evaluate_model(test_loader, test_df, "📈 Test Set Forecast vs True", "test")

📊 📈 Train Set Forecast vs True
 - MSE   : 0.000664
 - MAE   : 0.020301
 - SMAPE : 8.33%
✅ Plot saved to ../../outputs/metrics/autoformer/train_plot.png
✅ Results saved to ../../outputs/metrics/autoformer/train_results.csv



C:\Users\BERAT\AppData\Local\Temp\ipykernel_2660\437124191.py:44: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.savefig(f"../../outputs/metrics/autoformer/{file_prefix}_plot.png")


📊 📈 Validation Set Forecast vs True
 - MSE   : 0.000059
 - MAE   : 0.006783
 - SMAPE : 5.91%
✅ Plot saved to ../../outputs/metrics/autoformer/val_plot.png
✅ Results saved to ../../outputs/metrics/autoformer/val_results.csv



C:\Users\BERAT\AppData\Local\Temp\ipykernel_2660\437124191.py:44: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.savefig(f"../../outputs/metrics/autoformer/{file_prefix}_plot.png")


📊 📈 Test Set Forecast vs True
 - MSE   : 0.000043
 - MAE   : 0.005701
 - SMAPE : 2.63%
✅ Plot saved to ../../outputs/metrics/autoformer/test_plot.png
✅ Results saved to ../../outputs/metrics/autoformer/test_results.csv



C:\Users\BERAT\AppData\Local\Temp\ipykernel_2660\437124191.py:44: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.savefig(f"../../outputs/metrics/autoformer/{file_prefix}_plot.png")
